# ML

In [ ]:
# test the effects of undersampling

from os.path import join as pjoin
import pandas as pd
import numpy as np
import sys
import collections
from utils.stats_utils import get_pearson_and_spearman_correlation
from utils.data_utils import k_fold_cross_validation_split
from models.conventional_ml_models import lasso_regression, ridge_regression, mlp, xgboost, random_forest
from sklearn.preprocessing import StandardScaler
import scipy
import pickle
import torch

import skorch
from models.conventional_ml_models import MLP
import os

datas = ['ml-dp-hek293t-pe2.csv', 'ml-pd-hek293t-pe2.csv', 'ml-pd-adv-pe2.csv', 'ml-pd-k562-pe2.csv', 'ml-pd-k562mlh1d-pe2.csv']

performance = {}

def undersample(features, target):
    # sample 10% of the training data with editing efficiency of less than 10
    indices = np.where(target < 10)[0]
    seed = 42
    np.random.seed(seed)
    indices = np.random.choice(indices, int(len(indices) * 0.1), replace=False)
    features = np.delete(features, indices, axis=0)
    target = np.delete(target, indices, axis=0)
    return features, target

for data in datas:
    dataset = pd.read_csv(pjoin('models', 'data', 'conventional-ml', data))
    cell_line = '-'.join(data.split('-')[1:3]).split('.')[0]
    data_source = '-'.join(data.split('-')[1:]).split('.')[0]

    # 5 fold cross validation
    fold = 5

    features = dataset.iloc[:, :24].values
    target = dataset.iloc[:, -2].values

    # standardize the features
    # scaler = StandardScaler()
    # features = scaler.fit_transform(features)

    correlations = collections.defaultdict(list)
    
    for i in range(0, fold):
        print(f'Fold {i+1} of {fold}')
        train_features = features[dataset['fold'] != i]
        train_target = target[dataset['fold'] != i]
        
        # undersample the training data
        train_features, train_target = undersample(train_features, train_target)
        
        # ============================
        # Lasso
        # ============================
        print('Training Lasso')
        lasso_model = lasso_regression(train_features, train_target)
        lasso_model.fit(train_features, train_target)
        # pickle the model
        pickle.dump(lasso_model, open(pjoin('models', 'trained-models', 'conventional-ml', f'lasso-{data_source}-fold-{i+1}.pkl'), 'wb'))

        # ============================
        # Ridge
        # ============================
        print('Training Ridge')
        ridge_model = ridge_regression(train_features, train_target)
        ridge_model.fit(train_features, train_target)
        # pickle the model
        pickle.dump(ridge_model, open(pjoin('models', 'trained-models', 'conventional-ml', f'ridge-{data_source}-fold-{i+1}.pkl'), 'wb'))

        # ============================
        # xgboost
        # ============================
        print('Training XGBoost')
        xgboost_model = xgboost(train_features, train_target)
        xgboost_model.fit(train_features, train_target)
        # pickle the model
        pickle.dump(xgboost_model, open(pjoin('models', 'trained-models', 'conventional-ml', f'xgboost-{data_source}-fold-{i+1}.pkl'), 'wb'))

        # ============================
        # random forest
        # ============================
        print('Training Random Forest')
        rf_model = random_forest(train_features, train_target)
        rf_model.fit(train_features, train_target)
        # pickle the model
        pickle.dump(rf_model, open(pjoin('models', 'trained-models', 'conventional-ml', f'random_forest-{data_source}-fold-{i+1}.pkl'), 'wb'))
        
        # ============================
        # MLP
        # ============================
        
        print('Training MLP')
        mlp_model = skorch.NeuralNetRegressor(
            module=MLP,
            criterion=torch.nn.MSELoss,
            optimizer=torch.optim.Adam,
            max_epochs=200,
            module__activation='relu',
            lr=0.005,
            device='cuda',
            batch_size=2048,
            train_split=skorch.dataset.ValidSplit(cv=5),
            module__hidden_layer_sizes = (64, 64,),
            # early stopping
            callbacks=[
                skorch.callbacks.EarlyStopping(patience=30),
                skorch.callbacks.Checkpoint(monitor='valid_loss_best', f_pickle=None, f_criterion=None, f_optimizer=os.path.join('models', 'trained-models', 'conventional-ml', f'mlp-{data_source}-fold-{i+1}-optimizer.pkl'), f_history=os.path.join('models', 'trained-models', 'conventional-ml', f'mlp-{data_source}-fold-{i+1}-history.json'), f_params=os.path.join('models', 'trained-models', 'conventional-ml', f'mlp-{data_source}-fold-{i+1}.pkl'), event_name='event_cp'),
                skorch.callbacks.LRScheduler(policy=torch.optim.lr_scheduler.CosineAnnealingWarmRestarts, monitor='valid_loss', T_0=20, T_mult=1),
            ]
        )
        # convert features and target to float32
        train_features = train_features.astype(np.float32)
        train_target = train_target.astype(np.float32)
        # add a dimension to the target
        train_target = train_target[:, np.newaxis]
        mlp_model.fit(train_features, train_target)

In [ ]:
# load the pickled models and evaluate on the corresponding validation set
from os.path import join as pjoin
import pandas as pd
import numpy as np
import sys
import collections
from utils.stats_utils import get_pearson_and_spearman_correlation
from utils.data_utils import k_fold_cross_validation_split
from models.conventional_ml_models import lasso_regression, ridge_regression, mlp, xgboost, random_forest
from sklearn.preprocessing import StandardScaler
import scipy
import pickle
import torch
import skorch

datas = ['ml-dp-hek293t-pe2.csv', 'ml-pd-hek293t-pe2.csv', 'ml-pd-adv-pe2.csv', 'ml-pd-k562-pe2.csv', 'ml-pd-k562mlh1d-pe2.csv']

performance = {}

for data in datas:
    dataset = pd.read_csv(pjoin('models', 'data', 'conventional-ml', data))
    cell_line = '-'.join(data.split('-')[1:3]).split('.')[0]
    
    data_source = '-'.join(data.split('-')[1:]).split('.')[0]

    # 5 fold cross validation
    fold = 5

    features = dataset.iloc[:, :24].values
    target = dataset.iloc[:, -2].values

    correlations = collections.defaultdict(list)
    
    for i in range(0, fold):
        X_test = features[dataset['fold'] == i]
        y_test = target[dataset['fold'] == i]
        print(f'Fold {i+1} of {fold}')

        # ============================
        # Lasso
        # ============================
        lasso_model = pickle.load(open(pjoin('models', 'trained-models', 'conventional-ml', f'lasso-{data_source}-fold-{i+1}.pkl'), 'rb'))
        lasso_pred = lasso_model.predict(X_test)
        pearson, spearman = get_pearson_and_spearman_correlation(y_test, lasso_pred)
        correlations['Lasso'].append((pearson, spearman))
        print(f'Lasso: Pearson: {pearson}, Spearman: {spearman}')

        # ============================
        # Ridge
        # ============================
        ridge_model = pickle.load(open(pjoin('models', 'trained-models', 'conventional-ml', f'ridge-{data_source}-fold-{i+1}.pkl'), 'rb'))
        ridge_pred = ridge_model.predict(X_test)
        pearson, spearman = get_pearson_and_spearman_correlation(y_test, ridge_pred)
        correlations['Ridge'].append((pearson, spearman))
        print(f'Ridge: Pearson: {pearson}, Spearman: {spearman}')
        
        # ============================
        # xgboost
        # ============================
        xgboost_model = pickle.load(open(pjoin('models', 'trained-models', 'conventional-ml', f'xgboost-{data_source}-fold-{i+1}.pkl'), 'rb'))
        xgboost_pred = xgboost_model.predict(X_test)
        pearson, spearman = get_pearson_and_spearman_correlation(y_test, xgboost_pred)
        correlations['XGBoost'].append((pearson, spearman))
        print(f'XGBoost: Pearson: {pearson}, Spearman: {spearman}')
        
        # ============================
        # random forest
        # ============================
        rf_model = pickle.load(open(pjoin('models', 'trained-models', 'conventional-ml', f'random_forest-{data_source}-fold-{i+1}.pkl'), 'rb'))
        rf_pred = rf_model.predict(X_test)
        pearson, spearman = get_pearson_and_spearman_correlation(y_test, rf_pred)
        correlations['RF'].append((pearson, spearman))
        print(f'Random Forest: Pearson: {pearson}, Spearman: {spearman}')
        
        # ============================
        # MLP
        # ============================
        # load the pickled model
        mlp_model = skorch.NeuralNetRegressor(
            module=MLP,
            criterion=torch.nn.MSELoss,
            optimizer=torch.optim.Adam,
            max_epochs=200,
            module__activation='relu',
            lr=0.001,
            device='cuda',
            batch_size=2048,
            module__hidden_layer_sizes = (64, 64,),
        )
        mlp_model.initialize()
        mlp_model.load_params(
            f_optimizer=os.path.join('models', 'trained-models', 'conventional-ml', f'mlp-{data_source}-fold-{i+1}-optimizer.pkl'), 
            f_history=os.path.join('models', 'trained-models', 'conventional-ml', f'mlp-{data_source}-fold-{i+1}-history.json'), 
            f_params=os.path.join('models', 'trained-models', 'conventional-ml', f'mlp-{data_source}-fold-{i+1}.pkl')
        )
        X_test = X_test.astype(np.float32)
        X_test = torch.tensor(X_test)
        mlp_pred = mlp_model.predict(X_test)
        # flatten the prediction
        mlp_pred = mlp_pred.flatten()
        pearson, spearman = get_pearson_and_spearman_correlation(y_test, mlp_pred)
        correlations['MLP'].append((pearson, spearman))
        print(f'MLP: Pearson: {pearson}, Spearman: {spearman}')
        
        print('-----------------------------------')
    
    print(correlations)
    performance[cell_line] = correlations

# save the performance
save_file = pjoin('models', 'data', 'performance', 'conventional-ml-performance.csv')
performance_df = pd.DataFrame(performance)
print(performance_df)
performance_df.to_csv(save_file)
print(f'Performance saved to {save_file}')

In [ ]:
# plot the performance in two heat maps, one for pearson and one for spearman
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import ast

from os.path import join as pjoin

# cell line vs model
performance_data = pjoin('models', 'data', 'performance', 'conventional-ml-performance.csv')
df = pd.read_csv(performance_data, index_col=0)

cell_lines = df.columns
models = df.index

# plot the mean of the pearson and spearman correlation
pearson = np.zeros((len(models), len(cell_lines)))
spearman = np.zeros((len(models), len(cell_lines)))

for i, model in enumerate(models):
    for j, cell_line in enumerate(cell_lines):
        performance = df.loc[model, cell_line]
        # string to list
        performance = ast.literal_eval(performance)
        pearson[i, j] = np.mean([x[0] for x in performance])
        spearman[i, j] = np.mean([x[1] for x in performance])

font_size = 18

# pearson and spearman correlation share the same color bar
fig, axes = plt.subplots(1, 2, figsize=(14, 7), width_ratios=[1, 1])
sns.heatmap(pearson, ax=axes[0], annot=True, xticklabels=cell_lines, yticklabels=models, cmap='icefire', cbar=False, vmin=0, vmax=1, annot_kws={'size': font_size})
axes[0].set_title('Pearson')
sns.heatmap(spearman, ax=axes[1], annot=True, xticklabels=cell_lines, yticklabels=models, cmap='icefire', cbar=False, vmin=0, vmax=1, annot_kws={'size': font_size})
axes[1].set_title('Spearman')


# increase the font size
for ax in axes:
    for item in ([ax.title, ax.xaxis.label, ax.yaxis.label] + ax.get_xticklabels() + ax.get_yticklabels()):
        item.set_fontsize(font_size)
# # increase color bar font size
# cbar = axes[1].collections[0].colorbar
# cbar.ax.tick_params(labelsize=font_size)
# cbar.set_label('Correlation', fontsize=font_size)


# rotate the x labels
for ax in axes:
    ax.set_xticklabels(ax.get_xticklabels(), rotation=45, horizontalalignment='right')
# rotate the y labels
for ax in axes:
    ax.set_yticklabels(ax.get_yticklabels(), rotation=0, horizontalalignment='right')
# make sure the figure saved is not cut off
plt.tight_layout()

# save the figure
plt.savefig(pjoin('dissertation', 'figures', 'conventional_ml_models_performance.png'))

# DeepPrime

In [1]:
'''
Training DeepPrime
'''
from os.path import join as pjoin, basename

from models.deepprime import DeepPrime, preprocess_deep_prime, train_deep_prime

fname = pjoin('dp-pd-hek293t-pe2.csv')
train_deep_prime(fname, hidden_size=128, num_features=24, num_layers=1, dropout=0.05, epochs=500, batch_size=1024, lr=0.005, patience=10, device='cuda', num_runs=5, adjustment='none')    

Index(['wt-sequence', 'mut-sequence', 'pbs-length', 'rt-length',
       'extension-length', 'edit-length', 'rha-length', 'edit-position',
       'edit-type-replacement', 'edit-type-insertion', 'edit-type-deletion',
       'pbs-melting-temperature', 'rtt-wt-cdna-melting-temperature',
       'rtt-wt-cdna-new-melting-temperature', 'rtt-cdna-melting-temperature',
       'rtt-melting-temperature', 'delta-melting-temperature',
       'pbs-gc-content', 'rtt-gc-content', 'extension-gc-content',
       'pbs-gc-count', 'rtt-gc-count', 'extension-gc-count',
       'extension-minimum-free-energy', 'spacer-minimum-free-energy',
       'spcas9-score', 'group-id', 'editing-efficiency', 'fold'],
      dtype='object')
Fold 1 of 5
Training DeepPrime model...
Run 1 of 5
  epoch    train_loss    valid_loss    cp     dur
-------  ------------  ------------  ----  ------
      1     1346.4830     1799.0755     +  1.1482
      2      987.9143     1076.5977     +  0.6170
      3      771.1346     1028.3992   

In [ ]:
'''
Testing DeepPrime on the clinvar dataset
'''
from os.path import join as pjoin, basename

from models.deepprime import DeepPrime, preprocess_deep_prime, train_deep_prime, predict_deep_prime

data = 'dp-pd-hek293t-pe2.csv'
predict_deep_prime(data, hidden_size=128, num_layers=1, dropout=0.05, num_features=24, adjustment='none')

In [ ]:
# test log and undersample adjustments on deep prime
from models.deepprime import DeepPrime, preprocess_deep_prime, train_deep_prime, predict_deep_prime
import pandas as pd

datas = ['dp-dp-hek293t-pe2.csv', 'dp-pd-hek293t-pe2.csv', 'dp-pd-adv-pe2.csv', 'dp-pd-k562-pe2.csv', 'dp-pd-k562mlh1d-pe2.csv']

for data in datas:        
    print('Training Log')
    deepprime_log = train_deep_prime(data, hidden_size=128, num_features=24, num_layers=1, dropout=0.05, epochs=500, batch_size=2048, lr=0.005, patience=20, device='cuda', adjustment='log')
    
    print('Training Undersample')
    deepprime_undersample = train_deep_prime(data, hidden_size=128, num_features=24, num_layers=1, dropout=0.05, epochs=500, batch_size=2048, lr=0.005, patience=20, device='cuda', adjustment='undersample')
    
    print('Training Org')
    deepprime_org = train_deep_prime(data, hidden_size=128, num_features=24, num_layers=1, dropout=0.05, epochs=500, batch_size=2048, lr=0.005, patience=20, device='cuda')
        
    print('-----------------------------------')

In [ ]:
# test log and undersample adjustments on deep prime
from models.deepprime import DeepPrime, preprocess_deep_prime, train_deep_prime, predict_deep_prime
from os.path import join as pjoin
import pandas as pd

datas = ['dp-pd-hek293t-pe2.csv', 'dp-pd-adv-pe2.csv', 'dp-pd-k562-pe2.csv', 'dp-pd-k562mlh1d-pe2.csv', 'dp-dp-hek293t-pe2.csv']

performance_pearson = {}
performance_spearman = {}


for data in datas:
    print('-----------------------------------')
    print('Log adjustment')
    # log
    deepprime_log_pred, deepprime_log_performance = predict_deep_prime(data, hidden_size=128, num_layers=1, dropout=0.05, num_features=24, adjustment='log')
    
    print('-----------------------------------')
    print('Undersample adjustment')
    # undersample
    deepprime_undersample_pred, deepprime_undersample_performance = predict_deep_prime(data, hidden_size=128, num_layers=1, dropout=0.05, num_features=24, adjustment='undersample')
    
    print('-----------------------------------')
    print('Original')
    # original
    deepprime_org_pred, deepprime_og_performance = predict_deep_prime(data, hidden_size=128, num_layers=1, dropout=0.05, num_features=24)
    
    # save the predictions
    performance_pearson[data.split('.')[0]] = {
        'orginal': [x[0] for x in deepprime_og_performance],
        'log': [x[0] for x in deepprime_log_performance],
        'undersample': [x[0] for x in deepprime_undersample_performance]
    }
    
    performance_spearman[data.split('.')[0]] = {
        'orginal': [x[1] for x in deepprime_og_performance],
        'log': [x[1] for x in deepprime_log_performance],
        'undersample': [x[1] for x in deepprime_undersample_performance]
    }
    
# save the performance
performance_pearson_df = pd.DataFrame(performance_pearson)
performance_spearman_df = pd.DataFrame(performance_spearman)

# invert the x and y axis
performance_pearson_df = performance_pearson_df.T
performance_spearman_df = performance_spearman_df.T

performance_pearson_df.to_csv(pjoin('models', 'data', 'performance', 'deep-prime-performance-pearson.csv'))
performance_spearman_df.to_csv(pjoin('models', 'data', 'performance', 'deep-prime-performance-spearman.csv'))

print('Performance saved')

In [ ]:
# plot both the pearson and spearman correlation performance
# each figure contains the performance of the model on the different datasets
import pandas as pd
import numpy as np
from os.path import join as pjoin
import matplotlib.pyplot as plt
import seaborn as sns
import ast
import scipy

performance_pearson = pd.read_csv(pjoin('models', 'data', 'performance', 'deep-prime-performance-pearson.csv'), index_col=0)
performance_spearman = pd.read_csv(pjoin('models', 'data', 'performance', 'deep-prime-performance-spearman.csv'), index_col=0)
f_size = 14

adjustment_shorthands = {
    'weighted-mse': '   WMSE',
    'quantile-transform': '     QT',
    'original': '     OG',
    'log': '     LA',
    'undersample': '     US'
}

# convert all list strings to list
import ast

performance_pearson = performance_pearson.map(lambda x: ast.literal_eval(x))
performance_spearman = performance_spearman.map(lambda x: ast.literal_eval(x))

for i, data in enumerate(['dp-dp-hek293t-pe2', 'dp-pd-hek293t-pe2', 'dp-pd-k562-pe2', 'dp-pd-k562mlh1d-pe2', 'dp-pd-adv-pe2']):
    # plot the performance of the model on the different datasets
    fig, ax = plt.subplots(1, 2, figsize=(10, 1.8), width_ratios=(1.2, 1))
    cell_data_pearson = performance_pearson.loc[data]
    # into a dataframe of two columns, adjustment and performance
    cell_data_pearson = pd.DataFrame(np.array(cell_data_pearson.tolist()).T, columns=['original', 'log', 'undersample'])
    
    cell_data_spearman = performance_spearman.loc[data]
    cell_data_spearman = pd.DataFrame(np.array(cell_data_spearman.tolist()).T, columns=['original', 'log', 'undersample'])
    
    for ind, performance in enumerate([performance_pearson, performance_spearman]):
        # plot the mean of each adjustment as barplot
        # plot stripplot of the performance of each fold on top
        # of the barplot
        cell_data = cell_data_pearson if ind == 0 else cell_data_spearman
        sns.barplot(data=cell_data, ax=ax[ind], palette=iter(sns.color_palette('icefire', 3)),alpha=0.5, orient='h', errorbar=None)
        sns.stripplot(data=cell_data, ax=ax[ind], palette=iter(sns.color_palette('icefire', 3)), size=5, jitter=True, orient='h')
        # ax[ind].set_title('Pearson' if ind == 0 else 'Spearman')
        metric = 'Pearson' if ind == 0 else 'Spearman'
        ax[ind].set_xlabel(f'{metric}', fontsize=f_size)
        # ax[ind].set_ylabel('Adjustment')
        
        # fix y axis to 0 to 1
        ax[ind].set_xlim(0, 1)
        
        # remove top and right spines
        ax[ind].spines['top'].set_visible(False)
        if ind == 0:
            ax[ind].spines['left'].set_visible(False)
        else:
            ax[ind].spines['right'].set_visible(False)
            
        # test the significance between each adjustment with the original
        # using the paired t-test
        original = cell_data['original']
        for adj in cell_data.columns:
            if adj != 'original':
                if np.mean(cell_data[adj]) < np.mean(cell_data['original']):
                    t_stat, p_val = scipy.stats.ttest_rel(cell_data[adj], original)
                    if p_val < 0.05:
                        print(f'{data} {metric} {adjustment_shorthands[adj]}: {p_val}')
    
    ax[0].set_yticks(range(len(cell_data.columns)))
    ax[0].set_yticklabels([adjustment_shorthands[adj] for adj in cell_data.columns])
    # ax 1 should have no y labels
    ax[1].set_yticklabels([])
    # flip the x axis of ax 0
    ax[0].invert_xaxis()
    # move the y axis of ax 0 to the right
    ax[0].yaxis.tick_right()
    
    # change the font of x tick labels
    ax[0].tick_params(axis='both', which='major', labelsize=f_size)
    ax[1].tick_params(axis='both', which='major', labelsize=f_size)
        
    plt.tight_layout()
    
    # save the figure
    data_name = '-'.join(data.split('-')[1:3])
    plt.savefig(pjoin('dissertation/figures', f'adjustment-deepprime-{data_name}-performance.png'), dpi=300)

# PRIDICT

In [ ]:
from models.pridict import train_pridict

train_pridict('pd-pd-hek293t-pe2.csv', lr=0.005, batch_size=2048, epochs=200, patience=20, num_runs=5, adjustment='none', num_features=24)

Fold 1 of 5
Index(['wt-sequence', 'mut-sequence', 'edit-length', 'pbs-length',
       'rtt-length', 'rha-length', 'mut-type', 'edit-melting-temperature',
       'extension-melting-temperature', 'preedit-melting-temperature',
       'protospacer-melting-temperature', 'rha-melting-temperature',
       'pbs-melting-temperature', 'edit-sequence-zero-length',
       'preedit-sequence-zero-length', 'protospacer-location',
       'pbs-location-l-relative-protospacer',
       'rtt-location-l-relative-protospacer',
       'rha-location-l-relative-protospacer', 'extension-minimum-free-energy',
       'extension-scaffold-minimum-free-energy', 'pbs-minimum-free-energy',
       'spacer-minimum-free-energy',
       'spacer-extension-scaffold-minimum-free-energy',
       'spacer-scaffold-minimum-free-energy', 'rtt-minimum-free-energy'],
      dtype='object')
Training PRIDICT model...
Run 1 of 5
Re-initializing module.
Re-initializing criterion.
Re-initializing optimizer.
  epoch    train_loss    vali

In [ ]:
from models.pridict import predict_pridict

predict_pridict('pd-pd-hek293t-pe2.csv', num_features=24, device='cuda', adjustment='none')